# Gradient tap automatic differentiation 

https://github.com/sasidhar-programmer/Tensorflow_Advance_Techniques/blob/main/2-custom_and_distributed_training/week-1/C2_W1_Lab_2_gradient-tape-basics.ipynb

In [2]:
import tensorflow as tf

In [9]:
x = tf.ones([2,2])
print("x =",x)

with tf.GradientTape() as tape:
    tape.watch(x) # record actions performed on tensor x as it is 'watched' if we dont do this the result of taking the gradient with respect to x will be None
    y = tf.reduce_sum(x) # y = 4
    print("\ny =", y)
    z = tf.square(y) # z = y^2 = 16
    print("\nz =", z)

dz_dx = tape.gradient(z,x) # derive z with respect to x
# refer to course2_how_it_works_and_distributed.md to understand how the gradient was obtained
print("\ndz/dx = ", dz_dx)

x = tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)

y = tf.Tensor(4.0, shape=(), dtype=float32)

z = tf.Tensor(16.0, shape=(), dtype=float32)

dz/dx =  None


In [12]:
# the tape is good for only one derivation, if you want to use more than one then
# set peristannt =true
x= tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(x)
    y = x * x # y=x^2
    z = y*y # z=y^2

#dz/dx = 4 * x^3 = 108
dz_dx = t.gradient(z,x)
print(dz_dx)

tf.Tensor(108.0, shape=(), dtype=float32)


In [14]:
# now we get an error if we try to use the gradient tape t again since it is not persistant
try:
    dy_dx = t.gradient(y,x) # dy/dx = 2x = 6
    print(dy_dx)
except RuntimeError as e:
    print(e)

GradientTape.gradient can only be called once on non-persistent tapes.


In [17]:
# we set it to persistant now and can perform the gradient with the tape as many times as we want

# but we will need to manually delete the tape after we're done with it
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x * x # y=x^2
    z = y*y # z=y^2

#dz/dx = 4 * x^3 = 108
dz_dx = t.gradient(z,x)
print(dz_dx)

dy_dx = t.gradient(y,x) # dy/dx = 2x = 6
print(dy_dx)

del t

tf.Tensor(108.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


In [22]:
# higher order derivatives
# the first grad should be computed inside one of the with blocks
# but the second grad should be indented as much as the first grad or less
# ie it should be either in an outer block or in the same block of grad1
x = tf.Variable(1.0)
with tf.GradientTape() as tape2:
    with tf.GradientTape()as tape1:
        y= x*x*x
    dy_dx = tape1.gradient(y,x)


d2y_dx2 = tape2.gradient(dy_dx,x)
print(dy_dx)
print(d2y_dx2)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
